# Toy example to demonstrate the use of **mescal**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from mescal import *
import bw2data as bd
from utils import *
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
ei_version = '3.10.1'

In [ ]:
# AMPL licence
path_to_ampl_licence = r'C:\Users\matth\ampl' # Path to the AMPL license file
os.environ['PATH'] = path_to_ampl_licence+':'+os.environ['PATH']

In [ ]:
path_model = './data/esm/' # Path to the energy system model
path_model_lca = './data/esm/lca/'
path_inputs = './data/lca/' # Path to the LCA data
path_results = './results/' # Path to the results

## Generating LCA impact scores

In [ ]:
mapping = pd.read_csv(path_inputs+'mapping.csv')
unit_conversion = pd.read_excel(path_inputs+'unit_conversion.xlsx')
techno_compositions = pd.read_csv(path_inputs+'technology_compositions.csv')
efficiency = pd.read_csv(path_inputs+'efficiency.csv')
lifetime = pd.read_csv(path_inputs+'lifetime.csv')
mapping_es_flows_to_cpc = pd.read_csv(path_inputs+'mapping_esm_flows_to_CPC.csv')
impact_abbrev = pd.read_csv(path_inputs+'impact_abbrev.csv')
model = pd.read_csv(path_inputs+'model.csv')

In [ ]:
df_si_paper = pd.merge(mapping, unit_conversion, on=['Name', 'Type']).drop(columns=['Unit', 'Database']).rename(columns={'LCA': 'LCA unit', 'ESM': 'ESM unit', 'Name': 'Technology', 'Product': 'LCI dataset product', 'Activity': 'LCI dataset activity', 'Location': 'LCI dataset location', 'Value': 'Conversion factor', 'Type': 'Life cycle phase'})
df_si_paper['Technology'] = df_si_paper['Technology'].apply(lambda x: tech_name_dict[x] if x in tech_name_dict.keys() else x)
df_si_paper['Technology'] = df_si_paper['Technology'].replace('WIND_ONSHORE_CONNECTION', 'Onshore wind (connection part)')
df_si_paper['Technology'] = df_si_paper['Technology'].replace('WIND_ONSHORE_TURBINE', 'Onshore wind (turbine part)')
df_si_paper.to_csv('./data/lca/si_paper.csv', index=False)

In [ ]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ei_version}')

In [ ]:
name_main_database = f'ecoinvent_cutoff_{ei_version}_image_SSP2-Base_2050'

In [ ]:
main_db = Database(name_main_database, create_pickle=True)

In [ ]:
# Add CPC categories to the main database
main_db.add_CPC_categories()

In [ ]:
ranking_best_ecoinvent_locations = ['GLO', 'RoW']

In [ ]:
esm = ESM(
    # Mandatory inputs
    mapping=mapping,
    unit_conversion=unit_conversion,
    model=model,
    mapping_esm_flows_to_CPC_cat=mapping_es_flows_to_cpc,
    main_database=main_db,
    esm_db_name='Tatooine_2050',

    # Optional inputs
    technology_compositions=techno_compositions,
    lifetime=lifetime,
    efficiency=efficiency,
    regionalize_foregrounds=False,
    locations_ranking=ranking_best_ecoinvent_locations,
    esm_location='GLO',
    results_path_file=path_results,
)

In [ ]:
esm.check_inputs()

In [ ]:
# Adapt mapping file to ESM location
esm.change_location_mapping_file()
# esm.mapping.to_csv(path_inputs+f'mapping.csv', index=False)

In [ ]:
missing_flows = main_db.test_mapping_file(esm.mapping)

In [ ]:
esm.create_esm_database()

In [ ]:
# Save the mapping file with new codes for later use
esm.mapping.to_csv(path_results+'mapping_with_new_codes.csv', index=False)

In [ ]:
methods = ['IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10', 'IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10']

In [ ]:
R_long = esm.compute_impact_scores(
    methods=methods,
    impact_abbrev=impact_abbrev,
)

In [ ]:
R_long.to_csv(f'{path_results}impact_scores.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]

In [ ]:
activities_subject_to_double_counting = pd.read_csv(f'{path_results}activities_subject_to_double_counting.csv')

In [ ]:
R_long_direct_emissions = esm.compute_impact_scores(
    methods=methods,
    assessment_type='direct emissions',
    impact_abbrev=impact_abbrev,
    activities_subject_to_double_counting=activities_subject_to_double_counting,
    overwrite=True,
)

In [ ]:
R_long_direct_emissions.to_csv(f'{path_results}impact_scores_direct_emissions.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]

In [ ]:
metadata = {
    'ecoinvent_version': ei_version,
    'year': '2050',
    'iam': 'image',
    'ssp_rcp': 'SSP2-Base',
}

In [ ]:
specific_lcia_abbrev = ['m_CCS', 'TTHH', 'TTEQ']

In [ ]:
# Create .dat file
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_methods=methods,
    specific_lcia_abbrev=specific_lcia_abbrev,
    impact_abbrev=impact_abbrev,
    path=path_model_lca,
    metadata=metadata,
    file_name='techs_lca',
)

In [ ]:
# Create .dat file for direct emissions only
esm.normalize_lca_metrics(
    assessment_type='direct emissions',
    R=R_long_direct_emissions,
    max_per_cat=pd.read_csv(path_model_lca + 'techs_lca_max.csv'),
    mip_gap=1e-6,
    lcia_methods=methods,
    specific_lcia_abbrev=specific_lcia_abbrev,
    impact_abbrev=impact_abbrev,
    path=path_model_lca,
    metadata=metadata,
    file_name='techs_lca_direct',
)

In [ ]:
# Create the .mod file
esm.generate_mod_file_ampl(
    lcia_methods=methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=specific_lcia_abbrev,
    path=path_model_lca,
    metadata=metadata,
    file_name='objectives_lca',
)

In [ ]:
# Create the .mod file for direct emissions only
esm.generate_mod_file_ampl(
    assessment_type='direct emissions',
    lcia_methods=methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=specific_lcia_abbrev,
    path=path_model_lca,
    metadata=metadata,
    file_name='objectives_lca_direct',
)

## Running the ESM

In [ ]:
impact_scores = pd.read_csv(path_results+'impact_scores.csv')

### Single objective optimization

In [ ]:
list_esm_results_f_mult = []
list_esm_results_annual_res = []
list_esm_results_annual_prod = []
list_main_variables_results = []

for obj in ['TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']:

    results = run_esm(obj)

    df_f_mult, df_annual_prod, df_annual_res = get_impact_scores(
        df_results=results,
        df_impact_scores=impact_scores,
        impact_category=[
            ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10', 'Human health', 'Total human health'),
            ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10', 'Ecosystem quality', 'Total ecosystem quality'),
            ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10', 'Midpoint', 'Climate change, short term'),
        ]
    )

    df_f_mult['Run'] = obj
    df_annual_prod['Run'] = obj
    df_annual_res['Run'] = obj

    df_f_mult.rename(columns={'index': 'Name'}, inplace=True)
    df_annual_prod.rename(columns={'index': 'Name'}, inplace=True)
    df_annual_res.rename(columns={'index': 'Name'}, inplace=True)

    total_cost = results.variables['TotalCost'].TotalCost.values[0]
    total_m_ccs = results.variables['TotalLCIA_m_CCS'].TotalLCIA_m_CCS.values[0]
    total_tthh = results.variables['TotalLCIA_TTHH'].TotalLCIA_TTHH.values[0]
    total_tteq = results.variables['TotalLCIA_TTEQ'].TotalLCIA_TTEQ.values[0]

    list_main_variables_results.append([obj, total_cost, total_m_ccs, total_tthh, total_tteq])

    list_esm_results_f_mult.append(df_f_mult)
    list_esm_results_annual_prod.append(df_annual_prod)
    list_esm_results_annual_res.append(df_annual_res)

esm_results_f_mult = pd.concat(list_esm_results_f_mult)
esm_results_annual_prod = pd.concat(list_esm_results_annual_prod)
esm_results_annual_res = pd.concat(list_esm_results_annual_res)
main_variables_results = pd.DataFrame(data=list_main_variables_results, columns=['Objective', 'TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ'])

In [ ]:
esm_results_f_mult.to_csv('./results/soo_results_f_mult.csv', index=False)
esm_results_annual_prod.to_csv('./results/soo_results_annual_prod.csv', index=False)
esm_results_annual_res.to_csv('./results/soo_results_annual_res.csv', index=False)

In [ ]:
# lyrio = results.parameters['layers_in_out'].reset_index()
# lyrio = lyrio[lyrio.layers_in_out != 0].drop(columns=['Run']).rename({'index0': 'Name', 'index1': 'Flow', 'layers_in_out': 'Amount'}, axis=1)
# lyrio.to_csv(path_inputs+'model.csv', index=False)

### Multi-objective optimization

#### Pareto front

In [ ]:
# normalized limit = (physical limit [impact / cap] / max_AoP) * N_cap
N_run = 300
obj1 = 'TotalLCIA_m_CCS'
obj2 = 'TotalCost'

normalized_limits_list = []
obj1_min = main_variables_results[main_variables_results['Objective'] == obj1][obj1].values[0]
obj1_max = main_variables_results[main_variables_results['Objective'] == obj2][obj1].values[0]
physical_limit_list = list(np.linspace(obj1_min, obj1_max, N_run))
# physical_limit_list = list(np.logspace(np.log10(688.41), np.log10(750), N_run))
for limit in physical_limit_list:
        normalized_limits_list.append(limit)

In [ ]:
data = ['limit_lcia', obj1.split('TotalLCIA_')[-1], None, None, None] + normalized_limits_list
columns = ['param', 'index0', 'index1', 'index2', 'index3'] + [f'value{i+1}' for i in range(N_run)]

In [ ]:
seq_data = pd.DataFrame(data).T
seq_data.columns = columns

In [ ]:
es = run_esm(objective_function=obj2, returns='model')

In [ ]:
results_pareto = es.calc_sequence(seq_data)

In [ ]:
results_pareto = postprocessing(results_pareto)

#### Sobol sequence

In [ ]:
from energyscope.datasets import gen_sobol_sequence

In [ ]:
parameters = [
    {
        'name': cat,
        'lower_bound': main_variables_results[f'TotalLCIA_{cat}'].min(),
        'upper_bound': main_variables_results[f'TotalLCIA_{cat}'].max()
    }
    for cat in ['TTHH', 'TTEQ']
]

In [ ]:
seq, prob = gen_sobol_sequence(parameters=parameters, trajectories=128)

In [ ]:
seq_df = pd.DataFrame(seq, columns=prob['names']).T
seq_df.columns = ['value' + str(x) for x in list(seq_df.columns) if not str(x) == "nan"]
seq_df = seq_df.reset_index(names=['index0'])
seq_df['param'] = ['limit_lcia'] * len(seq_df)

seq_df['index1'] = np.nan
seq_df['index2'] = np.nan
seq_df['index3'] = np.nan

In [ ]:
es = run_esm(objective_function='TotalCost', returns='model')

In [ ]:
results_sobol = es.calc_sequence(seq_df)

In [ ]:
results_sobol = postprocessing(results_sobol)

## Visualize the results

In [ ]:
save_fig = True

In [ ]:
tech_to_show_list = [i for i in impact_scores.Name.unique() if i not in ['GRID']]

### SOO

In [ ]:
main_variables_results_norm = pd.DataFrame()
for col in main_variables_results.columns:
    if col != 'Objective':
        main_variables_results_norm[col] = main_variables_results[col] / main_variables_results[col].max()
    else:
        main_variables_results_norm['Objective function'] = main_variables_results[col]

In [ ]:
main_variables_results_norm.rename(columns=obj_name_dict, inplace=True)
main_variables_results_norm['Objective function'] = main_variables_results_norm['Objective function'].apply(lambda x: obj_name_dict[x])

In [ ]:
main_variables_results_norm = main_variables_results_norm.melt(id_vars=['Objective function'], var_name='Indicator', value_name='Normalized indicator')

In [ ]:
fig = px.bar(
    main_variables_results_norm,
    x='Objective function',
    y='Normalized indicator',
    color='Indicator',
    barmode='group',
)

fig.update_layout(template='plotly_white')

fig.show()

In [ ]:
fig = px.bar(
    main_variables_results_norm,
    color='Objective function',
    y='Normalized indicator',
    x='Indicator',
    barmode='group',
)

fig.update_layout(template='plotly_white')

fig.show()

In [ ]:
plot_technologies_contribution(
    cat='Total human health',
    esm_results_f_mult=esm_results_f_mult,
    esm_results_annual_prod=esm_results_annual_prod,
    esm_results_annual_res=esm_results_annual_res,
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

In [ ]:
plot_technologies_contribution(
    cat='Total ecosystem quality',
    esm_results_f_mult=esm_results_f_mult,
    esm_results_annual_prod=esm_results_annual_prod,
    esm_results_annual_res=esm_results_annual_res,
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

In [ ]:
plot_technologies_contribution(
    cat='Climate change, short term',
    esm_results_f_mult=esm_results_f_mult,
    esm_results_annual_prod=esm_results_annual_prod,
    esm_results_annual_res=esm_results_annual_res,
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

In [ ]:
plot_technologies_contribution(
    cat='Total cost',
    esm_results_f_mult=esm_results_f_mult,
    esm_results_annual_prod=esm_results_annual_prod,
    esm_results_annual_res=esm_results_annual_res,
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

### MOO

In [ ]:
plt.rcParams['font.family'] = 'arial'

#### Pareto front

In [ ]:
colors_var = 'TotalLCIA_TTHH'
add_soo_point = False

In [ ]:
x = [i * 1e3 * max_ccs / N_cap for i in physical_limit_list]
y = list(results_pareto.variables[obj2][obj2])
colors = list(results_pareto.variables[colors_var][colors_var])

# adding data point from colors_var SOO
if add_soo_point:
    x = x + [main_variables_results[main_variables_results['Objective'] == colors_var][obj1].values[0]]
    y = y + [main_variables_results[main_variables_results['Objective'] == colors_var][obj2].values[0]]
    colors = colors + [main_variables_results[main_variables_results['Objective'] == colors_var][colors_var].values[0]]

# Create the scatter plot
scatter = plt.scatter(x, y, c=colors, cmap='jet')

# Add a colorbar
cbar = plt.colorbar(scatter)
cbar.set_label(colors_var)

# Add labels and title
plt.xlabel(obj1)
plt.ylabel(obj2)

# Show the plot
plt.show()

In [ ]:
x = [i * 1e3 * max_ccs / N_cap for i in physical_limit_list]
y_1 = list(results_pareto.variables['TotalCost']['TotalCost'])
y_2 = list(results_pareto.variables['TotalLCIA_TTHH']['TotalLCIA_TTHH'])
y_3 = list(results_pareto.variables['TotalLCIA_TTEQ']['TotalLCIA_TTEQ'])
y_4 = list(results_pareto.variables['TotalLCIA_m_CCS']['TotalLCIA_m_CCS'])

# Normalize the y values using the max value
y_1 = [i / max(y_1) for i in y_1]
y_2 = [i / max(y_2) for i in y_2]
y_3 = [i / max(y_3) for i in y_3]
y_4 = [i / max(y_4) for i in y_4]

plt.figure(figsize=(4.5, 3.5))

plt.plot(x, y_1, label='Total cost', c='k')
plt.plot(x, y_2, label='Total human health', c='k', ls='dashed')
plt.plot(x, y_3, label='Total ecosystem quality', c='k', ls='dotted')
# plt.plot(x, y_4, label='Climate change, short term', c='k', ls='dashdot')

plt.xlabel(f'Upper limit for {obj_name_dict[obj1].lower()} [t CO$_2$-eq/cap]')
plt.ylabel('Normalized indicator')

plt.legend()
plt.tight_layout()
# plt.savefig('./figures/pareto_front_indicators.pdf')
plt.show()

In [ ]:
df_f_mult_pareto = results_pareto.variables['F_Mult'].reset_index()
x = [i * 1e3 * max_ccs / N_cap for i in physical_limit_list]
y_1 = list(df_f_mult_pareto[df_f_mult_pareto['index'] == 'CCGT']['F_Mult'])
y_2 = list(df_f_mult_pareto[df_f_mult_pareto['index'] == 'CCGT_CC']['F_Mult'])
y_3 = list(df_f_mult_pareto[df_f_mult_pareto['index'] == 'PV']['F_Mult'])

# from GW to kW / cap
y_1 = [i * 1e6 / N_cap for i in y_1]
y_2 = [i * 1e6 / N_cap for i in y_2]
y_3 = [i * 1e6 / N_cap for i in y_3]

plt.figure(figsize=(4.5, 3.5))

plt.plot(x, y_1, label=tech_name_dict['CCGT'], c=color_dict[tech_name_dict['CCGT']])
plt.plot(x, y_2, label=tech_name_dict['CCGT_CC'], c=color_dict[tech_name_dict['CCGT']], ls='-.')
plt.plot(x, y_3, label=tech_name_dict['PV'], c=color_dict[tech_name_dict['PV']])

plt.xlabel(f'Upper limit for {obj_name_dict[obj1].lower()} [t CO$_2$-eq/cap]')
plt.ylabel('Installed capacity [kW/cap]')

plt.legend()
plt.tight_layout()
# plt.savefig('./figures/pareto_front_capacities.pdf')
plt.show()

#### Sobol sequence

In [ ]:
def plot_heat_map_sobol(
        results_sobol,
        x_var: str,
        y_var: str,
        color_var: str,
        fill: bool,
):

    limit_lcia = results_sobol.parameters['limit_lcia']

    x=limit_lcia[limit_lcia.index == x_var].limit_lcia.tolist()
    y=limit_lcia[limit_lcia.index == y_var].limit_lcia.tolist()
    c=results_sobol.variables[color_var][color_var].tolist()

    if fill:
        plt.tricontourf(x, y, c, 128, cmap=plt.get_cmap('jet'))

    else:
        plt.scatter(
            x=limit_lcia[limit_lcia.index == x_var].limit_lcia.tolist(),
            y=limit_lcia[limit_lcia.index == y_var].limit_lcia.tolist(),
            c=results_sobol.variables[color_var][color_var].tolist(),
            cmap='jet',
        )

    # Add a colorbar
    cbar = plt.colorbar()
    cbar.set_label(color_var)

    plt.xlabel('Upper limit for ' + x_var)
    plt.ylabel('Upper limit for ' + y_var)

    plt.show()

In [ ]:
plot_heat_map_sobol(
    results_sobol=results_sobol,
    x_var = 'TTHH',
    y_var = 'TTEQ',
    color_var = 'TotalCost',
    fill=True,
)

### LCA metrics

In [ ]:
plot = Plot(
    df_impact_scores=impact_scores,
    lifetime=lifetime,
)

In [ ]:
plot.plot_indicators_of_technologies_for_one_impact_category(
    technologies_list=tech_to_show_list,
    impact_category=('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10', 'Human health', 'Total human health'),
    contributions_total_score=True,
)

In [ ]:
plot.plot_indicators_of_technologies_for_one_impact_category(
    technologies_list=tech_to_show_list,
    impact_category=('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10', 'Ecosystem quality', 'Total ecosystem quality'),
    contributions_total_score=True,
)